Start with imports.

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from pystan import StanModel
from keras.datasets import mnist
from vae import VAE
from gp import GP
from scipy.stats import multivariate_normal as mvn
from scipy.spatial.distance import cdist

In [9]:
import sys
print(sys.executable)

/Users/martins/Dev/py3/bin/python


In [13]:
stan_code = """

/*
Sample variance hyperparameters from the GP posterior distribution,
given a pre-specified set of kernel matrices.
*/

data {
  int<lower=1> N;       // Number of data points
  int<lower=1> D;       // Input space dimensionality
  matrix[N, D] X;       // Input design matrix
  vector[N] y;          // Target vector

  // Gamma hyperparameters
  real mu_noise;
  real sd_noise;
  real mu_signal;
  real sd_signal;
  real mu_band;
  real sd_band;
}

transformed data {
  vector[N] mu;
  for (i in 1:N) mu[i] = mean(y);
}

parameters {
  real log_bandwidth;
  real log_noise;
  real log_signal;
}

transformed parameters{
    real bandwidth;
    real noise;
    real signal;
    bandwidth = exp(log_bandwidth);
    noise = exp(log_noise);
    signal = exp(log_signal);
}

model {
    matrix[N, N] Sigma;
    log_noise ~ normal(mu_noise, sd_noise);
    log_signal ~ normal(mu_signal, sd_signal);
    log_bandwidth ~ normal(mu_band, sd_band);

    for (i in 1:N){
        for (j in 1:N){
            Sigma[i, j] = signal * exp(-bandwidth * square(dot_self(X[i] - X[j])));
        }
    }
    Sigma = Sigma + diag_matrix(rep_vector(noise, N));

    // GP
    y ~ multi_normal(mu, Sigma);
}
"""

In [ ]:
sm = StanModel(model_code=stan_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_7bba7403f204af5f998047409576fd7b NOW.


In [7]:
def exponential_kernel(X, Y, bandwidth=1.0):
    """
        :param X: Array of shape (n_samples, n_features)
        :param Y: Array of shape (n_samples, n_features)
        :param bandwidth: (``float``) Bandwidth.
        :return: Kernel matrix between data points.
    """
    return np.exp(-bandwidth * cdist(X, Y, metric="euclidean") ** 2)